In [1]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np
mx.random.seed(1) 

C:\Users\594084\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\contrib\pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
ctx = mx.cpu()


In [3]:
def transformer(data, label):
    data = mx.image.imresize(data, 224, 224)
    data = mx.nd.transpose(data, (2,0,1))
    data = data.astype(np.float32)
    return data, label


In [4]:
#Original batch size was 64, changed it to 3 to run faster
batch_size = 3
train_data = gluon.data.DataLoader(
    gluon.data.vision.CIFAR10('./data', train=True, transform=transformer),
    batch_size=batch_size, shuffle=True, last_batch='discard')

test_data = gluon.data.DataLoader(
    gluon.data.vision.CIFAR10('./data', train=False, transform=transformer),
    batch_size=batch_size, shuffle=False, last_batch='discard')

In [5]:

for d, l in train_data:
    print(d,l)
    if d == 3:
        break


[[[[ 101.  101.  101. ...,   45.   45.   45.]
   [ 101.  101.  101. ...,   45.   45.   45.]
   [ 101.  101.  101. ...,   45.   45.   45.]
   ..., 
   [ 138.  138.  138. ...,  117.  117.  117.]
   [ 138.  138.  138. ...,  117.  117.  117.]
   [ 138.  138.  138. ...,  117.  117.  117.]]

  [[  93.   93.   93. ...,   59.   59.   59.]
   [  93.   93.   93. ...,   59.   59.   59.]
   [  93.   93.   93. ...,   59.   59.   59.]
   ..., 
   [ 138.  138.  138. ...,   98.   98.   98.]
   [ 138.  138.  138. ...,   98.   98.   98.]
   [ 138.  138.  138. ...,   98.   98.   98.]]

  [[  77.   77.   77. ...,   28.   28.   28.]
   [  77.   77.   77. ...,   28.   28.   28.]
   [  77.   77.   77. ...,   28.   28.   28.]
   ..., 
   [ 112.  112.  112. ...,   85.   85.   85.]
   [ 112.  112.  112. ...,   85.   85.   85.]
   [ 112.  112.  112. ...,   85.   85.   85.]]]


 [[[ 216.  216.  216. ...,  115.  115.  115.]
   [ 216.  216.  216. ...,  115.  115.  115.]
   [ 216.  216.  216. ...,  115.  115.  115.

ValueError: The truth value of an NDArray with multiple elements is ambiguous.

In [6]:
print(d.shape)

(3, 3, 224, 224)


In [7]:
d.dtype


numpy.float32

In [8]:
alex_net = gluon.nn.Sequential()
with alex_net.name_scope():
    #  First convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=96, kernel_size=11, strides=(4,4), activation='relu'))
    alex_net.add(gluon.nn.MaxPool2D(pool_size=3, strides=2))
    #  Second convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=192, kernel_size=7, activation='relu'))
    alex_net.add(gluon.nn.MaxPool2D(pool_size=3, strides=(2,2)))
    # Third convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=384, kernel_size=5, activation='relu'))
    # Fourth convolutional layer
    #alex_net.add(gluon.nn.Conv2D(channels=384, kernel_size=3, activation='relu'))
    # Fifth convolutional layer
    #alex_net.add(gluon.nn.Conv2D(channels=256, kernel_size=3, activation='relu'))
    #alex_net.add(gluon.nn.MaxPool2D(pool_size=3, strides=2))
    # Flatten and apply fullly connected layers
    alex_net.add(gluon.nn.Flatten())
    alex_net.add(gluon.nn.Dense(4096, activation="relu"))
    alex_net.add(gluon.nn.Dense(4096, activation="relu"))
    alex_net.add(gluon.nn.Dense(10))

In [9]:
alex_net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

In [10]:
trainer = gluon.Trainer(alex_net.collect_params(), 'sgd', {'learning_rate': .001})
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()


In [11]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for d, l in data_iterator:
        data = d.as_in_context(ctx)
        label = l.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [ ]:
###########################
#  Only one epoch so tests can run quickly, increase this variable to actually run
###########################
epochs = 1
smoothing_constant = .01


for e in range(epochs):
    for i, (d, l) in enumerate(train_data):
        data = d.as_in_context(ctx)
        label = l.as_in_context(ctx)
        with autograd.record():
            output = alex_net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])

        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

    test_accuracy = evaluate_accuracy(test_data, alex_net)
    train_accuracy = evaluate_accuracy(train_data, alex_net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))